In [1]:
import pandas as pd

In [8]:
df=pd.read_csv("./data/reviews/reviews_month_imputed.csv")

In [9]:
df

,city,category,place_name,date,note,review,sentiment_label,plateforme,text,month
0,Marrakech,hotel,Movenpick Hotel Mansour Eddahbi Marrakech,NaN,NaN,Naima and Sami were very friendly people who h...,positive,Tripadvisor,Naima and Sami were very friendly people who h...,09/2014
1,Marrakech,hotel,Movenpick Hotel Mansour Eddahbi Marrakech,NaN,NaN,Excellent stay at the conveniently situated Mo...,positive,Tripadvisor,Excellent stay at the conveniently situated Mo...,09/2018
2,Marrakech,hotel,Movenpick Hotel Mansour Eddahbi Marrakech,NaN,NaN,An impressive hall with EN-suite facilities an...,positive,Tripadvisor,An impressive hall with EN-suite facilities an...,09/2014
3,Marrakech,hotel,Movenpick Hotel Mansour Eddahbi Marrakech,NaN,NaN,We stayed at the Movenpick hotel where we met ...,positive,Tripadvisor,We stayed at the Movenpick hotel where we met ...,01/2019
4,Marrakech,hotel,Movenpick Hotel Mansour Eddahbi Marrakech,NaN,NaN,Hotel on top! Great location and with great se...,positive,Tripadvisor,Hotel on top! Great location and with great se...,07/2022
...,...,...,...,...,...,...,...,...,...,...
98972,Dakhla,hotel,Hôtel Mansour\nUne nouvelle fenêtre va s'ouvrir,12/2024,4.0/5,Très bien | Positive: L'hôtel est situé dans u...,positive,Booking,Très bien | Positive: L'hôtel est situé dans u...,12/2024
98973,Dakhla,hotel,Hôtel Mansour\nUne nouvelle fenêtre va s'ouvrir,11/2024,3.5/5,Fabuleux | Positive: Le personnel de l’hôtel e...,negative,Booking,Fabuleux | Positive: Le personnel de l’hôtel e...,11/2024
98974,Dakhla,hotel,Hôtel Mansour\nUne nouvelle fenêtre va s'ouvrir,10/2024,4.5/5,Bien | Positive: Pas grand chose | Negative: R...,positive,Booking,Bien | Positive: Pas grand chose | Negative: R...,10/2024
98975,Dakhla,hotel,Hôtel Mansour\nUne nouvelle fenêtre va s'ouvrir,10/2024,4.0/5,Dommage je suis arrivé en plein travaux | Posi...,neutral,Booking,Dommage je suis arrivé en plein travaux | Posi...,10/2024


In [25]:
# Feature 1 : Avg_Positive
# = Moyenne des scores de confiance des avis POSITIFS uniquement
avg_positive = df[df['sentiment_label'] == 'positive'].groupby('month')

# Feature 2 : Avg_Negative
# = Moyenne des scores de confiance des avis NÉGATIFS uniquement
avg_negative = df[df['sentiment_label'] == 'negative'].groupby('month')

# Feature 3 : weighted_sentiment
# = Moyenne de la polarité de TOUS les avis du mois
weighted_sentiment = df.groupby('month')

# Combiner les 3 features
monthly_features = pd.DataFrame({
    'Avg_Positive': avg_positive,
    'Avg_Negative': avg_negative,
    'weighted_sentiment': weighted_sentiment
}).reset_index()

# Remplir les NaN (mois sans avis positifs/négatifs)
#monthly_features = monthly_features.fillna(0)

ValueError: If using all scalar values, you must pass an index

In [19]:
# Agrégation mensuelle
monthly = df.groupby('month').agg({
    'sentiment_label': [
        ('positive', lambda x: (x=='positive').sum()),
        ('negative', lambda x: (x=='negative').sum()),
        ('neutral', lambda x: (x=='neutral').sum()),
        ('total', 'count')
    ]
})



# Variables finales : sentiment_score, sentiment_lag1, review_volume

In [21]:
monthly.describe()

sentiment_label                                        
             positive    negative       neutral         total
count      175.000000  175.000000    175.000000    175.000000
mean        58.971429    9.480000    497.131429    565.582857
std        290.016833   47.851163   2265.876063   2577.952012
min          1.000000    0.000000      0.000000      2.000000
25%          4.000000    0.000000      0.000000      4.000000
50%          7.000000    1.000000      0.000000      9.000000
75%         14.500000    2.500000      0.000000     18.000000
max       3563.000000  593.000000  21168.000000  25324.000000

In [15]:
# Calcul des 3 variables
monthly['sentiment_score'] = (monthly['positive'] - monthly['negative']) / monthly['total']
monthly['sentiment_lag1'] = monthly['sentiment_score'].shift(1)
monthly['review_volume'] = monthly['total']


KeyError: 'positive'